In [10]:
from selenium import webdriver
from selenium.webdriver.support.select import Select
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

driver = webdriver.Chrome()
driver.get("https://transparenciabrasil-cidadao.govbr.cloud/transparenciabrasil-cidadao/itabira/prefeitura/conteudo?urlMenu=https:%2F%2Fservicositabira.govbr.cloud%2Fpronimtb%2Findex.asp%3Facao%3D3%26item%3D6%26reestruturacao")


driver.implicitly_wait(10)

In [ ]:

# Encontra o elemento select
select_element = driver.find_element(By.ID, "cmbUnidadeGestora")

# Cria um objeto Select a partir do elemento
select = Select(select_element)

# Seleciona uma opção pelo valor
select.select_by_value("2")  # Altere para o valor desejado

In [ ]:


# Wait for the element with the name "txtDataInicial" to be present
wait = WebDriverWait(driver, 20)
data_inicial = wait.until(EC.presence_of_element_located((By.NAME, "txtDataInicial")))

# Set the value of the hidden element using JavaScript
driver.execute_script("arguments[0].value = '01032024';", data_inicial)

# Submit the form
data_inicial.submit()

In [ ]:
# Wait for the element with the name "txtDataInicial" to be visible
wait = WebDriverWait(driver, 20)
data_inicial = wait.until(EC.visibility_of_element_located((By.NAME, "txtDataInicial")))

data_inicial.send_keys("01032024")
data_inicial.send_keys(Keys.RETURN)
data_inicial.clear()

## Scripts

In [ ]:
from selenium import webdriver
import time

# Configurar o Selenium WebDriver (usando o Chrome)
driver = webdriver.Chrome()

# Abrir a página da API do Portal da Transparência
driver.get("https://www.transparencia.gov.br/swagger-ui.html")

# Aguardar o carregamento da página
time.sleep(3)

# Encontrar o link "APIs" e clicar nele
apis_link = driver.find_element_by_link_text("APIs")
apis_link.click()

# Aguardar o carregamento da página de APIs
time.sleep(3)

# Encontrar o link "Dados Abertos" e clicar nele
dados_abertos_link = driver.find_element_by_link_text("Dados Abertos")
dados_abertos_link.click()

# Aguardar o carregamento da página de Dados Abertos
time.sleep(3)

# Encontrar o link "Consultas" e clicar nele
consultas_link = driver.find_element_by_link_text("Consultas")
consultas_link.click()

# Aguardar o carregamento da página de Consultas
time.sleep(3)

# Encontrar o link "Execução Orçamentária" e clicar nele
execucao_orcamentaria_link = driver.find_element_by_link_text("Execução Orçamentária")
execucao_orcamentaria_link.click()

# Aguardar o carregamento da página de Execução Orçamentária
time.sleep(3)

# Encontrar o botão "Consultar" e clicar nele
consultar_button = driver.find_element_by_xpath("//button[contains(text(), 'Consultar')]")
consultar_button.click()

# Aguardar o carregamento dos dados
time.sleep(5)

# Imprimir a URL atual (que contém o endpoint da API)
print("URL do Endpoint da API:", driver.current_url)

# Fechar o navegador
driver.quit()
